# Answer 3

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
def recover_bases(pile):
    """
    Parse sequence pile to ignore mapping qualities
    and indels. Return sequence of just matches/mismatches.
    """
    
    # Remove $ and ^x from string
    pile = re.sub(r"\^.", "", pile.replace("$", ""))
    
    # Remove indels from string (none in demo data)
    pile = re.sub(r"\+\d+[ACGT]+", "", pile)
    pile = re.sub(r"\-\d+[ACGT]+", "", pile)
    
    return pile

Read in pileup data

In [3]:
with open("../../data/question_003/pileup/BL6_x_Cast_RNA.pileup", "r") as f:
    pileup_data = [line.split("\t") for line in f]

Print reference chromosome, reference position, and consensus base if:

* The sequence coverage of quality base reads is greater than or equal to 5X and less than 100X


* Mismatches relative to the reference is greater than or equal to 80% of the total number of bases in the pile

All other reads are ignored.


In [4]:
results = []

for entry in pileup_data[:]:

    reference_chromosome = entry[0].strip()
    reference_position = int(entry[1].strip())
    reference_base = entry[2].strip()
    sequence_pile = recover_bases(entry[4].strip())
    sequence_quality = entry[5].strip()
        
    # Exclude bases with quality score less than 40
    pile_quality_zipped =  zip(list(sequence_pile),
                               [ord(q) - 33 for q in sequence_quality])
    
    zipped_filtered = [x for x in pile_quality_zipped if x[1] >= 40]
    
    if zipped_filtered:
        # Reconstruct sequence
        bases_filtered = "".join([x[0].upper() for x in zipped_filtered])
        
        # Calculate sequence coverage only from bases with high quality
        sequence_coverage = len(bases_filtered)
        
        # Skip entry if coverage less than 5 of greater than/equal to 100
        if sequence_coverage < 5 or sequence_coverage >= 100:
            continue
        
        # If sequence all the same base, add consensus base to results
        if len(set(bases_filtered)) == 1:
            results.append({
                    "reference_chromosome": reference_chromosome,
                    "reference_position": reference_position,
                    "consensus_base": bases_filtered[0]
                })
        else:
            # Calculate if base is more than 80% of sequence
            base_count = {base for base in set(bases_filtered) 
                          if bases_filtered.count(base) 
                          / len(bases_filtered) >= 0.80}
            # Print consensus sequence if it is
            if base_count:
                results.append({
                    "reference_chromosome": reference_chromosome,
                    "reference_position": reference_position,
                    "consensus_base":  list(base_count)[0]
                })

In [5]:
consensus_bases = pd.DataFrame(results)

In [6]:
consensus_bases[["reference_chromosome", "reference_position", "consensus_base"]]

reference_chromosome  reference_position consensus_base
0                    chr7           147526773              G
1                    chr7           147526774              T
2                    chr7           147526775              T
3                    chr7           147526776              T
4                    chr7           147526777              G
5                    chr7           147526778              T
6                    chr7           147526779              A
7                    chr7           147526780              G
8                    chr7           147526781              G
9                    chr7           147526782              C
10                   chr7           147526784              C
11                   chr7           147526785              T
12                   chr7           147526786              A
13                   chr7           147526787              G
14                   chr7           147526791              A
15                   chr7           147526792              G
16                   chr7           147526793              G
17                   chr7           147526794              A
18                   chr7           147526796              T
19                   chr7           147526798              G
20                   chr7           147526799              G
21                   chr7           147526800              G
22                   chr7           147526802              T
23                   chr7           147526803              G
24                   chr7           147526804              G
25                   chr7           147526805              G
26                   chr7           147526807              T
27                   chr7           147571616              G
28                   chr7           147571617              G
29                   chr7           147571618              C
...                   ...                 ...            ...
3153                 chr7           147999171              T
3154                 chr7           147999172              C
3155                 chr7           147999173              C
3156                 chr7           147999174              T
3157                 chr7           147999175              T
3158                 chr7           147999176              C
3159                 chr7           147999177              T
3160                 chr7           147999178              T
3161                 chr7           147999179              T
3162                 chr7           147999180              G
3163                 chr7           147999181              A
3164                 chr7           147999182              T
3165                 chr7           147999183              G
3166                 chr7           147999184              A
3167                 chr7           147999185              A
3168                 chr7           147999186              A
3169                 chr7           147999187              A
3170                 chr7           147999188              G
3171                 chr7           147999189              C
3172                 chr7           147999190              C
3173                 chr7           147999191              T
3174                 chr7           147999192              T
3175                 chr7           147999193              T
3176                 chr7           147999194              G
3177                 chr7           147999195              G
3178                 chr7           147999197              A
3179                 chr7           147999198              C
3180                 chr7           147999202              G
3181                 chr7           147999203              A
3182                 chr7           147999814              T

[3183 rows x 3 columns]

In [7]:
consensus_bases.to_csv("consensus_bases.tsv", sep="\t")